In [1]:
import scipy.io as sio
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
from pyDOE import lhs
import torch.nn.functional as F
device = "cuda:1"

In [2]:
class fc(nn.Module):

    def __init__(self, num_classes=2):
        super(fc, self).__init__()
        self.fc1 = nn.Linear(2, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 100)
        self.fc4 = nn.Linear(100, 100)
        self.fc5 = nn.Linear(100, 100)
        self.out = nn.Linear(100, num_classes)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = F.tanh(self.fc4(x))
        x = F.tanh(self.fc5(x))
        x = self.out(x)
        return x

In [17]:
class PINN():
    def __init__(self, x0, u0_star, v0_star, tb, lb, ub, t_f, x_f):
        self.x0 = torch.tensor(x0, dtype=torch.float32, requires_grad=True).to(device)
        self.t0 = torch.zeros_like(self.x0, dtype=torch.float32, requires_grad=True).to(device)
        self.u0_star = torch.tensor(u0_star, dtype=torch.float32).to(device)
        self.v0_star = torch.tensor(v0_star, dtype=torch.float32).to(device)
        self.tb = torch.tensor(tb, dtype=torch.float32, requires_grad=True).to(device)
        self.lb = torch.tensor(lb, dtype=torch.float32, requires_grad=True).to(device)
        self.ub = torch.tensor(ub, dtype=torch.float32, requires_grad=True).to(device)
        self.t_f = torch.tensor(t_f, dtype=torch.float32, requires_grad=True).to(device)
        self.x_f = torch.tensor(x_f, dtype=torch.float32, requires_grad=True).to(device)

        self.criterion = torch.nn.MSELoss(reduction="mean")
        self.fc = fc().to(device)
        self.optimizer = torch.optim.LBFGS(
          self.fc.parameters(),
          lr=1.0,
          max_iter=50000,
          max_eval=50000,
          history_size=50,
          tolerance_grad=1e-5,
          tolerance_change=1.0 * np.finfo(float).eps,
          line_search_fn="strong_wolfe"       # can be "strong_wolfe"
        )
        self.optimizer_adam = torch.optim.Adam(self.fc.parameters(),lr=0.001)
        self.iter = 0

    
    def grad(self, h, x):
        return torch.autograd.grad(h, x, grad_outputs=torch.ones_like(h), create_graph=True, retain_graph=True)

    def latent_solution_h(self, x, t):
        input_xt = torch.concat((x, t), dim=1)
        uv = self.fc(input_xt)
        u, v = uv[:, [0]], uv[:, [1]]
        return u, v
    
    def shrodinger_equation(self, x, t):
        u, v = self.latent_solution_h(x, t)
        u_x = self.grad(u, x)[0]
        u_t = self.grad(u, t)[0]
        u_xx = self.grad(u_x, x)[0]
        v_x = self.grad(v, x)[0]
        v_xx = self.grad(v_x, x)[0]
        v_t = self.grad(v, t)[0]
        real = -v_t + 0.5*u_xx + (u**2 + v**2)*u
        imag = u_t + 0.5*v_xx + (u**2 + v**2)*v
        return real, imag

    def loss_function(self):
        self.optimizer.zero_grad()
        u0_pred, v0_pred = self.latent_solution_h(self.x0, self.t0)
        loss_initial_data = self.criterion(u0_pred, self.u0_star) + self.criterion(v0_pred, self.v0_star)
        u_tb_lb_pred, v_tb_lb_pred = self.latent_solution_h(self.lb, self.tb)
        u_tb_ub_pred, v_tb_ub_pred = self.latent_solution_h(self.ub, self.tb)
        u_tb_lb_pred_x = self.grad(u_tb_lb_pred, self.lb)[0]
        u_tb_ub_pred_x = self.grad(u_tb_ub_pred, self.ub)[0]
        v_tb_lb_pred_x = self.grad(v_tb_lb_pred, self.lb)[0]
        v_tb_ub_pred_x = self.grad(v_tb_ub_pred, self.ub)[0]
        loss_col_boundary = torch.mean((u_tb_lb_pred - u_tb_ub_pred)**2) + torch.mean((v_tb_lb_pred - v_tb_ub_pred)**2) + torch.mean((u_tb_lb_pred_x - u_tb_ub_pred_x)**2) + torch.mean((v_tb_lb_pred_x - v_tb_ub_pred_x)**2)
        real, imag = self.shrodinger_equation(self.x_f, self.t_f)
        loss_col_f = torch.mean(real**2) + torch.mean(imag**2)
        total_loss = loss_initial_data + loss_col_boundary + loss_col_f
        if self.iter % 50 ==0:
            print(f"Iter {self.iter}, Total Loss {total_loss}, Loss_0 {loss_initial_data}, Loss_b {loss_col_boundary}, Loss_f {loss_col_f}")
        self.iter += 1
        total_loss.backward()
        return total_loss
    
    def train(self, iter):
        self.fc.train()
        for i in range(iter):
            self.optimizer_adam.zero_grad()
            u0_pred, v0_pred = self.latent_solution_h(self.x0, self.t0)
            loss_initial_data = self.criterion(u0_pred, self.u0_star) + self.criterion(v0_pred, self.v0_star)
            u_tb_lb_pred, v_tb_lb_pred = self.latent_solution_h(self.lb, self.tb)
            u_tb_ub_pred, v_tb_ub_pred = self.latent_solution_h(self.ub, self.tb)
            u_tb_lb_pred_x = self.grad(u_tb_lb_pred, self.lb)[0]
            u_tb_ub_pred_x = self.grad(u_tb_ub_pred, self.ub)[0]
            v_tb_lb_pred_x = self.grad(v_tb_lb_pred, self.lb)[0]
            v_tb_ub_pred_x = self.grad(v_tb_ub_pred, self.ub)[0]
            loss_col_boundary = torch.mean((u_tb_lb_pred - u_tb_ub_pred)**2) + torch.mean((v_tb_lb_pred - v_tb_ub_pred)**2) + torch.mean((u_tb_lb_pred_x - u_tb_ub_pred_x)**2) + torch.mean((v_tb_lb_pred_x - v_tb_ub_pred_x)**2)
            real, imag = self.shrodinger_equation(self.x_f, self.t_f)
            loss_col_f = torch.mean(real**2) + torch.mean(imag**2)
            total_loss = loss_initial_data + loss_col_boundary + loss_col_f
            if self.iter % 50 ==0:
                print(f"Iter {self.iter}, Total Loss {total_loss}, Loss_0 {loss_initial_data}, Loss_b {loss_col_boundary}, Loss_f {loss_col_f}")
            self.iter += 1
            total_loss.backward()
            self.optimizer_adam.step()

        self.optimizer.step(self.loss_function)

    def predict(self, x, t):
        x = torch.tensor(x, requires_grad=True).float().to(device)
        t = torch.tensor(t, requires_grad=True).float().to(device)
        self.neural_network.eval()
        u, v = self.latent_solution_h(x, t)
        u_x = self.grad(u, x)[0]
        u_t = self.grad(u, t)[0]
        u_xx = self.grad(u_x, x)[0]
        v_x = self.grad(v, x)[0]
        v_xx = self.grad(v_x, x)[0]
        v_t = self.grad(v, t)[0]
        real = -v_t + 0.5*u_xx + (u**2 + v**2)*u
        imag = u_t + 0.5*v_xx + (u**2 + v**2)*v
        return u.detach().cpu(), v.detach().cpu(), real.detach().cpu(), imag.detach().cpu()




In [18]:
N_0 = 50
N_b = 50
N_f = 20000

lb = np.array([-5, 0])
ub = np.array([5, np.pi/2])
mat = sio.loadmat("./NLS.mat")
t = mat["tt"].reshape(-1, 1)
uu = mat["uu"]
x = mat["x"].reshape(-1, 1)
u = np.real(uu)
v = np.imag(uu)
h = np.sqrt(u**2 + v**2)
X, T = np.meshgrid(x, t)
x0 = x.copy()
tb = t.copy()
train_x0_idx = np.random.choice(x0.shape[0], N_0, replace=False)
train_x0 = x0[train_x0_idx, :]
train_u0 = u[train_x0_idx, 0:1]
train_v0 = v[train_x0_idx, 0:1]
print(train_x0.shape, train_u0.shape, train_v0.shape)
train_tb_idx = np.random.choice(tb.shape[0], N_b, replace=False)
train_tb = tb[train_tb_idx, :]
X_T_f = lb + (ub - lb)*lhs(2, N_f)

train_x_f = X_T_f[:, [0]]
train_t_f = X_T_f[:, [1]]
train_lb = -5 * np.ones_like(train_tb)
train_ub = 5 * np.ones_like(train_tb)

(50, 1) (50, 1) (50, 1)


In [20]:
model = PINN(train_x0, train_u0, train_v0, train_tb, train_lb, train_ub, train_t_f, train_x_f)
model.train(2000)

Iter 0, Total Loss 1.5770729780197144, Loss_0 1.5683625936508179, Loss_b 0.007978132925927639, Loss_f 0.0007323367753997445
Iter 50, Total Loss 0.5031263828277588, Loss_0 0.3572239279747009, Loss_b 0.005891059525310993, Loss_f 0.14001137018203735
Iter 100, Total Loss 0.2534375786781311, Loss_0 0.1448429971933365, Loss_b 0.0036616174038499594, Loss_f 0.10493295639753342
Iter 150, Total Loss 0.26084545254707336, Loss_0 0.1469908356666565, Loss_b 0.005065010394901037, Loss_f 0.1087896078824997
Iter 200, Total Loss 0.20705774426460266, Loss_0 0.1208416000008583, Loss_b 0.0014898664085194468, Loss_f 0.08472628891468048
Iter 250, Total Loss 0.20698288083076477, Loss_0 0.11655757576227188, Loss_b 0.0011861873790621758, Loss_f 0.08923912793397903
Iter 300, Total Loss 0.18868961930274963, Loss_0 0.10748899728059769, Loss_b 0.001375674968585372, Loss_f 0.07982493937015533
Iter 350, Total Loss 0.20134437084197998, Loss_0 0.12099366635084152, Loss_b 0.0027299770154058933, Loss_f 0.077620729804039


In [25]:
u_pred, v_pred, _, _, f_u_pred, f_v_pred = model.predict(x, t)

h_pred = np.sqrt(u_pred**2 + v_pred**2)
error_u = np.linalg.norm(u - u_pred, 2) / np.linalg.norm(u, 2)
error_v = np.linalg.norm(v - v_pred, 2) / np.linalg.norm(v, 2)
error_h = np.linalg.norm(h - h_pred, 2) / np.linalg.norm(h, 2)
print('Error u: %e' % (error_u))
print('Error v: %e' % (error_v))
print('Error h: %e' % (error_h))


(256, 201)

In [13]:
print(t.shape, x.shape, u.shape, v.shape)

(201, 1) (256, 1) (256, 201) (256, 201)


In [36]:
print(tb.shape)

(201, 1)


In [28]:
print(T[[0], :])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [30]:
print(x, t)

[[-5.       ]
 [-4.9609375]
 [-4.921875 ]
 [-4.8828125]
 [-4.84375  ]
 [-4.8046875]
 [-4.765625 ]
 [-4.7265625]
 [-4.6875   ]
 [-4.6484375]
 [-4.609375 ]
 [-4.5703125]
 [-4.53125  ]
 [-4.4921875]
 [-4.453125 ]
 [-4.4140625]
 [-4.375    ]
 [-4.3359375]
 [-4.296875 ]
 [-4.2578125]
 [-4.21875  ]
 [-4.1796875]
 [-4.140625 ]
 [-4.1015625]
 [-4.0625   ]
 [-4.0234375]
 [-3.984375 ]
 [-3.9453125]
 [-3.90625  ]
 [-3.8671875]
 [-3.828125 ]
 [-3.7890625]
 [-3.75     ]
 [-3.7109375]
 [-3.671875 ]
 [-3.6328125]
 [-3.59375  ]
 [-3.5546875]
 [-3.515625 ]
 [-3.4765625]
 [-3.4375   ]
 [-3.3984375]
 [-3.359375 ]
 [-3.3203125]
 [-3.28125  ]
 [-3.2421875]
 [-3.203125 ]
 [-3.1640625]
 [-3.125    ]
 [-3.0859375]
 [-3.046875 ]
 [-3.0078125]
 [-2.96875  ]
 [-2.9296875]
 [-2.890625 ]
 [-2.8515625]
 [-2.8125   ]
 [-2.7734375]
 [-2.734375 ]
 [-2.6953125]
 [-2.65625  ]
 [-2.6171875]
 [-2.578125 ]
 [-2.5390625]
 [-2.5      ]
 [-2.4609375]
 [-2.421875 ]
 [-2.3828125]
 [-2.34375  ]
 [-2.3046875]
 [-2.265625 ]
 [-2.2